In [19]:
from random import random
import time

def f(x):
    # start = time.time()
    time.sleep(x)
    # print(f'{time.time() - start} secs')
    return

print('Start')
start = time.time()
q = [random() for _ in range(100)]

for n in q:
    f(n)

print(f'End {time.time() - start} Secs')


Start
End 52.425944328308105 Secs


In [20]:
from multiprocessing import Pool

def fu(x):
    # start = time.time()
    time.sleep(x)
    # print(f'{time.time() - start} secs')
    return

print('Start')
start = time.time()

with Pool(processes=4) as pool:
    # q = [random() for _ in range(5)]
    pool.map(fu, [random() for _ in range(100)])

print(f'End {time.time() - start} Secs')

Start
End 14.153250455856323 Secs


In [10]:
queue = ['88445a5457fffff',
'88445a54e3fffff',
'88445a54e9fffff',
'88445a54ebfffff',
'88445a5601fffff',
'88445a5607fffff',
'88445a5647fffff',
'88445a566bfffff',
'8844e82821fffff']

In [27]:
import h3.api.numpy_int as h3
import requests

def get_loc_data(hex8):
    (lat,lon) = h3.h3_to_geo(h3.string_to_h3(hex8))
    
    params = {
        'x': lon,
        'y': lat,
        'benchmark': 'Public_AR_Current',
        'vintage': 'Current_Current',
        'format': 'json'
    }

    retries = 3
    complete = False

    while not complete:
        try:

            r = requests.get('https://geocoding.geo.census.gov/geocoder/geographies/coordinates', params=params).json()

            state = r['result']['geographies']['States'][0]['STUSAB']
            city = r['result']['geographies']['County Subdivisions'][0]['NAME']

        except Exception as e:
            print(e)
        else:
            complete = True
            return (city, state)
        finally:
            if not complete and retries:
                retries -= 1
            elif not complete and not retries:
                return None

In [47]:
def check(args):
    a = [*args]
    start = time.time()
    c_s = get_loc_data(a[1])
    return f'Task {a[0]}: {c_s} in {time.time() - start} secs\n'
    

In [ ]:
new_queue = queue * 10
with Pool(processes=4) as pool:
    start = time.time()
    imap = pool.imap(check, [(i, q) for i,q in enumerate(new_queue)], chunksize=10)
    
    for x in imap:
        print(x)

print(f'Complete {time.time() - start} secs')

In [45]:
new_queue = queue * 2
print(new_queue)

['88445a5457fffff', '88445a54e3fffff', '88445a54e9fffff', '88445a54ebfffff', '88445a5601fffff', '88445a5607fffff', '88445a5647fffff', '88445a566bfffff', '8844e82821fffff', '88445a5457fffff', '88445a54e3fffff', '88445a54e9fffff', '88445a54ebfffff', '88445a5601fffff', '88445a5607fffff', '88445a5647fffff', '88445a566bfffff', '8844e82821fffff']


In [57]:
import os
import csv
import h3.api.numpy_int as h3
import requests
from multiprocessing import Pool

queue = []
done_q = []

with open('../mt/csv/cable_202211281042.csv') as csv_file:
    csv_read = csv.reader(csv_file, delimiter=',')
    
    for i,row in enumerate(csv_read):
        if i == 0:
            continue
        queue.append((i, row[0]))
    

def run(args):
    i, h8 = args
    
    c_s = get_loc_data(h8)
    
    if c_s:
        city, state = c_s
        return (i, h8, city, state)
    else:
        return None


def get_loc_data(hex8):
    (lat,lon) = h3.h3_to_geo(h3.string_to_h3(hex8))
    
    params = {
        'x': lon,
        'y': lat,
        'benchmark': 'Public_AR_Current',
        'vintage': 'Current_Current',
        'format': 'json'
    }

    retries = 3
    complete = False

    while not complete:
        try:

            r = requests.get('https://geocoding.geo.census.gov/geocoder/geographies/coordinates', params=params).json()

            state = r['result']['geographies']['States'][0]['STUSAB']
            city = r['result']['geographies']['County Subdivisions'][0]['NAME']

        except Exception as e:
            print(e)
        else:
            complete = True
            return (city, state)
        finally:
            if not complete and retries:
                retries -= 1
            elif not complete and not retries:
                return None


with Pool(processes=4) as pool:
    imap = pool.imap(run, queue, chunksize=10)
    
    for m in imap:
        if not m:
            continue
        else:
            i, h8, city, state = m
            done_q.append((h8, city, state))
            
            if not i % 10:
                print(f'Task {i} complete \n')

with open('../mt/csv/first_100.csv', mode='w') as csv_file:
    csv_write = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    csv_write.writerow(['hex8', 'city', 'state'])
    
    for d in done_q:
        h8, city, state = d
        csv_write.writerow([h8, city, state])
            

Task 10 complete 

Task 20 complete 

Task 30 complete 

Task 40 complete 

Task 50 complete 

Task 60 complete 

Task 70 complete 

Task 80 complete 

Task 90 complete 

Task 100 complete 



In [53]:
import h3.api.numpy_int as h3
(lat,lon) = h3.h3_to_geo(h3.string_to_h3('88445a54ebfffff'))
# hex8 = h3.geo_to_h3(41.083064, -81.518485, 8)
# h3.h3_to_string(hex8)
# h3.h3_get_resolution(h3.string_to_h3('88445a5457fffff'))
(lat, lon)

(33.57544449136207, -88.27223597175066)

In [42]:
import os
import csv
import h3.api.numpy_int as h3
import requests
from multiprocessing import Pool
from IPython.display import clear_output


class MultiT:
    
    def __init__(self):
        self.queue = []
        self.done_q = []
        self.errors = []
        self.start = time.time()
        self.end = None
    
    def open_csv(self, f_name):

        with open(f_name) as csv_file:
            csv_read = csv.reader(csv_file, delimiter=',')

            for i,row in enumerate(csv_read):
                if i == 0:
                    continue
                self.queue.append((i, row[0]))
    

    def run(self, args):
        i, h8 = args

        c_s = self.get_loc_data(h8)

        if c_s:
            city, state = c_s
            return (i, h8, city, state)
        else:
            return None


    def get_loc_data(self, hex8):
        (lat,lon) = h3.h3_to_geo(h3.string_to_h3(hex8))

        params = {
            'lon': lon,
            'lat': lat,
            'format': 'geojson'
        }

        try:

            r = requests.get('http://localhost:8080/reverse.php', params=params).json()
            r = r['features'][0]['properties']

            state = r['address']['ISO3166-2-lvl4'][-2:]
            c_type = r['type']

            if r['address']['city']:
                city = r['address']['city']
            elif r['address']['town']:
                city = r['address']['town']
            else:
                city = r['address'][c_type]

        except Exception as e:
            self.errors.append(hex8)
            return None
        else:
            return (city, state)

    def pool(self, processes):

        with Pool(processes=processes) as pool:
            imap = pool.imap(self.run, self.queue, chunksize=10)

            for m in imap:
                if not m:
                    continue
                else:
                    i, h8, city, state = m
                    self.done_q.append((h8, city, state))

                    if not i % 10:
                        clear_output(wait=True)
                        print(f'Task {i} complete in {time.time() - self.start} secs')
        
        self.end = time.time()
    
    def write(self, f_name):

        with open(f_name, mode='w') as csv_file:
            csv_write = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            csv_write.writerow(['hex8', 'city', 'state'])

            for d in self.done_q:
                h8, city, state = d
                csv_write.writerow([h8, city, state])
        
        with open('../mt/csv/errors.log', 'w') as e_log:
            e_log.writelines(self.errors)
        
        print(f'Done in {self.end - self.start} secs with {len(self.errors)} errors')

In [43]:
mt = MultiT()
mt.open_csv('../mt/csv/200k-210k.csv')


In [44]:
mt.pool(2)

Task 9770 complete in 249.779620885849 secs


In [45]:
mt.write('../mt/csv/final_200k-210k.csv')

Done in 255.94881629943848 secs with 0 errors


In [52]:
'Asheville, NC'.split(',')

['Asheville', ' NC']